# Libraries import
- Data wrangling and indicators generate
- api to trade (Binance)
- api llm (openai)

In [53]:
import os
import pandas as pd
import numpy as np
import schedule
import ta
import time
import re
from datetime import datetime
from dotenv import load_dotenv
from binance.client import Client
import openai
from openai import OpenAI

load_dotenv();

# APIS Configuration

In [54]:
mclient = Client(os.getenv("BINANCE_API_KEY"), api_secret = os.getenv("BINANCE_API_SECRET"),testnet=True)

In [55]:
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# We build functions
- Data download
- Calculation of indicators
- Signals
- Operations executor
- Log function
- Agent executor

In [56]:
def get_data(ticker,period):  
    klines = mclient.get_historical_klines(ticker, period, limit = 1000) #since, to
    df = pd.DataFrame(klines)
    df = df.iloc[:,[0,1,2,3,4,5,7,8,9,10]] 
    df.columns = ['Time','Open','High','Low','Close','VolumeCurr','VolumeUSD','Trades','VolumeBase','Volume2']
    df = df.set_index('Time')
    df.index = pd.to_datetime(df.index, unit='ms')
    df=df.astype("float")
    df['ASSET'] = f"{ticker}"[0:3] 
    return df

In [57]:
def get_indicators(df):
    df['var_pct'] = df['Close'].pct_change()
    df = df.dropna(subset=['var_pct']).copy()  # Asegúrate de trabajar con una copia explícita
    df['SMA_20'] = ta.trend.sma_indicator(df['Close'], window=20)
    df['SMA_50'] = ta.trend.sma_indicator(df['Close'], window=50)
    df['RSI'] = ta.momentum.rsi(df['Close'], window=14)
    df['MACD'] = ta.trend.macd(df['Close'])
    df['MACD_Signal'] = ta.trend.macd_signal(df['Close'])

    df['MA20'] = df['Close'].rolling(20).mean()
    df['MA100'] = df['Close'].rolling(100).mean()
    
    df['Signal'] = 0.0
    df.loc[df['MA20'] > df['MA100'], 'Signal'] = 1.0
    df.loc[df['MA20'] < df['MA100'], 'Signal'] = -1.0

    df['Plog_ret'] = np.log(df.Close) - np.log(df.Close.shift(1))
    df['pup_6std'] = df.Close*(1+(df.Plog_ret.mean() + 6*df.Plog_ret.std()))
    df['pdown_6std'] = df.Close*(1+(df.Plog_ret.mean() - 6*df.Plog_ret.std()))

    df['std_dev'] = df['Close'].rolling(window=20).std()
    df['upper_band'] = df['SMA_20'] + (6 * df['std_dev'])
    df['lower_band'] = df['SMA_20'] - (6 * df['std_dev'])
    return df

In [58]:
def get_signal(df):

    buy_signals = (df['Signal'].shift(1) == -1.0) & (df['Signal'] == 1.0)
    sell_signals = (df['Signal'].shift(1) == 1.0) & (df['Signal'] == -1.0)

    df.loc[buy_signals, 'Trade_Signal'] = 'BUY'

    df.loc[sell_signals, 'Trade_Signal'] = 'SELL'
    return df

In [59]:
mbalance = mclient.get_asset_balance(asset='USDT')

In [60]:
def get_chat(system_prompt,user_message):
    completion = openai_client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
      {"role":'system','content':system_prompt},
      {"role": "user", "content": user_message}
    ]
  )
    respon = completion.choices[0].message.content
    masset = re.search(r"\*\*Asset\*\*: (\w+)",respon).group(1)
    direction = re.search(r"\*\*Direction\*\*: (\w+)",respon).group(1)
    if direction == 'Hold':
        msize = 0
    else:
        msize = re.search(r"\*\*Operation size\*\*: (\d+ USD)",respon).group(1)[0:3]
    print(respon)
    return masset, direction, msize

In [61]:
def get_trade(direccion, activo):
    global price_eject
    if masset == 'USDT':

        price_eject = 0
        print('No order is placed')
    
    else:
        symbol_to_find = f"{activo}"+'USDT'
        cantidad = round(100/float(next((item for item in mclient.get_all_tickers() if item['symbol'] == symbol_to_find), None)['price']),4)
        if direccion == 'Buy':
            order = mclient.order_market_buy(symbol=f"{activo}"+'USDT', quantity=cantidad,type = 'MARKET')
            price_eject = order['fills'][0]['price']
            print("Purchase order executed:", order)
        elif direccion == 'Sell':
            order = mclient.order_market_buy(symbol=f"{activo}"+'USDT', quantity=cantidad,type = 'MARKET')
            price_eject = order['fills'][0]['price']
            print("Sell ​​order executed:", order)
        else:
            price_eject = 0
        print('No order is placed')

In [62]:
def run_agent(ticker,period):
    data = get_data(ticker,period)
    data = get_indicators(data)
    data = get_signal(data)

    system_prompt = (
    '#Instructions: \n'
    'Act as an experienced cryptocurrency trader who helps users identify market trends.\n'
    'You help people who want to trade according to their strategy, however, they want an experts opinion regarding their decision to buy or sell, which is not based only on their strategy but also on your expertise.\n'
    'You are provided with a' + f"{mbalance}" + 'that represents the total amount available that the user has to trade, on the asset that the user provides you.\n'
    'Make the decision to buy or sell the asset according to the users strategy and also on the pool of technical indicators that will be shared with you.\n'
    'The value in USD of each individual operation must not exceed the' + f"{mbalance}" + 'and the operations must have a size that allows trading with what is available in' + f"{mbalance}" + 'and also allows handling market volatility.\n'
    'Do not suggest or provide a reasoning for the order when the suggested order size (both for new positions and for adding to existing positions) is less than 100 USD.\n'
    '# Available options:\n'
    '- close a position only when there is a moving average crossover opposite to the crossover with which a trade is opened\n'
    '- there must not be 2 open positions in the same week'
    '# Fields for each option:\n'
    '- asset: the asset to be trade is the' + f"{data['ASSET'].iloc[0]}" +'cryptocurrency\n'
    '- direction: the direction to trade\n'
    '\t- example: buy, sell\n'
    '- size: the size of the trade denominated in USD. It should be greater than 10 and should not use all of the' + f"{mbalance}" + 'leaving sufficient funds available to risk management.\n'
    '- reasoning: the reasoning for the decision\n'
    'if in the variable' + f"{data['Trade_Signal'].iloc[-1]}" + 'there is a BUY signal then you should take a long or buy position if the other indicators you check confirm it\n'
    'if in the variable' + f"{data['Trade_Signal'].iloc[-1]}" + 'there is a SELL signal then you should take a short or sell position if the other indicators you check confirm it'
    )
        
    user_message = (
    '# Instructions:\n'
    'Here are some details about me, can you help me make decisions about what position I should take?\n'
    'I need to place a trade weekly\n'
    '# Available Balance\n'
    'The available balance to trade is' + f"{mbalance}" + 'however each trade must be 100 USDT\n'
    'This' + f"{mbalance}" + 'can be used to place new orders or modify existing positions.\n'
    'Always leave a fraction of the' + f"{mbalance}" + 'total as a safety cushion for unforeseen volatility.\n'
    'The' + f"{mbalance}" + 'is shared by all positions, so its important to keep track of the available value and adjust your position sizes accordingly.\n'
    '# This is the most recent information I want to base my decisions on:\n'
    'My strategy is based on the crossover of moving averages, whose signal of trade that is ' + f"{data['Trade_Signal'].iloc[-1]}" + 'this can be BUY or SELL, but as I understand that this is a decision that can be influenced by other indicators, I ask you based on your experience to evaluate the following indicators\n'
    f"""
    Technical analysis data for the asset:
    - SMA 20: {data['SMA_20'].iloc[-1]}"
    - SMA 50: {data['SMA_50'].iloc[-1]}
    - RSI: {data['RSI'].iloc[-1]}
    - MACD: {data['MACD_Signal'].iloc[-1]}
    - Short mean reversion: {data['upper_band'].iloc[-1]}
    - Long mean reversion: {data['lower_band'].iloc[-1]}
    """
    'In addition to all the advice you give me, I need you to always summarize your conclusion with these points **Asset**, **Direction** (here the options are buy, sell, hold) and **Operation size**, in case it is to hold and a new position is not generated, then suggest 0 for size'    
    )

    masset, mdirection, msize = get_chat(system_prompt,user_message)
    get_trade(mdirection, masset[0:3])

In [64]:
run_agent('BTCUSDT','1h')

Based on your strategy and the provided technical indicators, let's analyze the situation.

1. **Moving Averages**: The SMA 20 is above the SMA 50 (101180.027 vs. 100119.475), which typically indicates a bullish sentiment. However, the gap suggests there might be caution as the current price trend may not be sustainably upward considering the short-term versus long-term perspective.

2. **RSI**: At 21.88, the RSI indicates the asset is oversold. Generally, an RSI below 30 suggests that the asset could potentially be undervalued, making it a candidate for a buy signal.

3. **MACD**: The MACD figure is relatively high (155.50), generally suggesting bullish momentum. However, without knowing the MACD signal line and historical values, it’s important to conclude that while momentum is present, it might not indicate an immediate strong buying opportunity without confirmation.

4. **Mean Reversion**: The short mean reversion at 108605.31 suggests a rebound target, while the long mean reversi

# Scheduled Agent Execution
This code runs a trading agent for BTCUSDT pair every minute, executing exactly 3 times before stopping. 
It uses a simple counter approach with the schedule library for precise timing control.

In [45]:
"""
for i in range(3):  # Will run 3 times
    run_agent('BTCUSDT', '1h')
    print(f"Execution {i+1} of 3 completed")
    if i < 2:  # Don't sleep after the last execution
        time.sleep(60)  # Wait 1 minute between executions
        
print("All 3 executions completed!")
"""

'\nfor i in range(3):  # Will run 3 times\n    run_agent(\'BTCUSDT\', \'1h\')\n    print(f"Execution {i+1} of 3 completed")\n    if i < 2:  # Don\'t sleep after the last execution\n        time.sleep(60)  # Wait 1 minute between executions\n        \nprint("All 3 executions completed!")\n'